<a href="https://colab.research.google.com/github/ashleysally00/Uttarakhand-Land-Surface-Temperature-LST-Analysis-Using-GEE/blob/main/Uttarakhand_LST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install earthengine-api geemap


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.4 MB/s eta 0:00:00


In [4]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-ashleysally00')


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [5]:
# Import Earth Engine
import ee

# Authenticate and Initialize Earth Engine
ee.Authenticate()
ee.Initialize()

# Load the shapefile from Earth Engine Assets
uttarakhand = ee.FeatureCollection("projects/ee-ashleysally00/assets/Uttarakhand-boundary")

# Print details about the shapefile (optional)
print(uttarakhand.getInfo())


{'type': 'FeatureCollection', 'columns': {'STATE': 'String', 'Shape_Area': 'Float', 'Shape_Leng': 'Float', 'State_LGD': 'Long', 'system:index': 'String'}, 'version': 1733066237497692, 'id': 'projects/ee-ashleysally00/assets/Uttarakhand-boundary', 'properties': {'system:asset_size': 152477}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[77.57231450945571, 30.41044935748887], [77.57233680954047, 30.409031342861944], [77.57236356963637, 30.40792548257725], [77.57236356963637, 30.407907658948393], [77.57262216278623, 30.40651200036795], [77.57325539431214, 30.404496502491888], [77.57391975452882, 30.402538901745867], [77.57456635858219, 30.400576957985976], [77.57478034750606, 30.400055178703944], [77.57479818718556, 30.400015101102994], [77.57479818718556, 30.400006188694793], [77.574802647105, 30.400006184934863], [77.57482048678101, 30.399957108370405], [77.57486508595869, 30.399841209322894], [77.57511484102889, 30.399212454292798], [77.575805949612

In [6]:
# Clip MODIS dataset to Uttarakhand boundary
modis_clipped = modis.map(lambda img: img.clip(uttarakhand))


NameError: name 'modis' is not defined

In [8]:
# Load the MODIS LST dataset
modis = ee.ImageCollection("MODIS/061/MOD11A1")


In [9]:
modis_clipped = modis.map(lambda img: img.clip(uttarakhand))


In [11]:
# Resample the MODIS data to 500m resolution
modis_resampled = modis_clipped.map(
    lambda img: img.resample('bilinear').reproject(crs='EPSG:4326', scale=500)
)


In [12]:
# Extract daytime LST and scale values
lst_day = modis_resampled.select('LST_Day_1km').map(
    lambda img: img.multiply(0.02).rename('LST_Day_K')
)


In [13]:
# Extract nighttime LST and scale values
lst_night = modis_resampled.select('LST_Night_1km').map(
    lambda img: img.multiply(0.02).rename('LST_Night_K')
)


In [14]:
# Calculate the mean daytime LST
mean_lst_day = lst_day.mean()

# Calculate the mean nighttime LST
mean_lst_night = lst_night.mean()


In [15]:
# Visualize mean daytime LST
import geemap
Map = geemap.Map()
Map.centerObject(uttarakhand, 8)
Map.addLayer(mean_lst_day, {'min': 250, 'max': 320, 'palette': ['blue', 'green', 'red']}, 'Mean Daytime LST')

# Visualize mean nighttime LST
Map.addLayer(mean_lst_night, {'min': 250, 'max': 320, 'palette': ['blue', 'green', 'red']}, 'Mean Nighttime LST')

# Display the map
Map


Map(center=[30.155046043180448, 79.210196652578], controls=(WidgetControl(options=['position', 'transparent_bg…

In [16]:
# Export mean daytime LST to Google Drive
ee.batch.Export.image.toDrive(
    image=mean_lst_day,
    description='Mean_LST_Day_Uttarakhand',
    folder='EarthEngineExports',
    fileNamePrefix='Mean_LST_Day',
    region=uttarakhand.geometry(),
    scale=500,
    crs='EPSG:4326',
    maxPixels=1e13
).start()

# Export mean nighttime LST to Google Drive
ee.batch.Export.image.toDrive(
    image=mean_lst_night,
    description='Mean_LST_Night_Uttarakhand',
    folder='EarthEngineExports',
    fileNamePrefix='Mean_LST_Night',
    region=uttarakhand.geometry(),
    scale=500,
    crs='EPSG:4326',
    maxPixels=1e13
).start()
